## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-11-18-57-11 +0000,nyt,Senate Deadlocks Over Obamacare Subsidies in D...,https://www.nytimes.com/2025/12/11/us/politics...
1,2025-12-11-18-54-15 +0000,nyt,U.S. Helped to Weaken Report at U.N. Environme...,https://www.nytimes.com/2025/12/11/climate/une...
2,2025-12-11-18-54-00 +0000,wsj,"Ukraine Pitches Trump on Vision for Peace, but...",https://www.wsj.com/world/europe/ukraine-pitch...
3,2025-12-11-18-53-09 +0000,nyt,"Mike Lindell, MyPillow Founder, Announces Run ...",https://www.nytimes.com/2025/12/11/us/politics...
4,2025-12-11-18-52-39 +0000,bbc,Thousands ready to evacuate as flooding hits P...,https://www.bbc.com/news/articles/ce91dergm00o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2363/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,40
475,new,19
682,fed,16
120,oil,14
283,venezuela,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
228,2025-12-11-04-00-00 +0000,wsj,President Trump wants lower interest rates and...,https://www.wsj.com/economy/central-banking/fe...,120
111,2025-12-11-14-17-42 +0000,nyt,Trump Gets to Pick a New Fed Chair but a Divid...,https://www.nytimes.com/2025/12/11/business/fe...,97
17,2025-12-11-18-27-00 +0000,wapo,U.S. seizes ‘very large’ oil tanker off Venezu...,https://www.washingtonpost.com/world/2025/12/1...,96
272,2025-12-10-23-56-02 +0000,nypost,Trump says he wants a new owner for CNN as par...,https://nypost.com/2025/12/10/media/trump-says...,94
213,2025-12-11-06-52-47 +0000,nyt,Trump Says U.S. Seized Oil Tanker Off Venezuel...,https://www.nytimes.com/2025/12/10/us/politics...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
228,120,2025-12-11-04-00-00 +0000,wsj,President Trump wants lower interest rates and...,https://www.wsj.com/economy/central-banking/fe...
300,78,2025-12-10-22-35-53 +0000,bbc,Watch: Video shows US military seizing oil tan...,https://www.bbc.com/news/videos/c9vj1jnmekko?a...
34,54,2025-12-11-17-51-00 +0000,wsj,The U.S. trade deficit narrowed to its lowest ...,https://www.wsj.com/economy/trade/u-s-trade-de...
25,52,2025-12-11-18-11-23 +0000,nypost,Olivia Nuzzi feared ‘Bobby will kill me’ if af...,https://nypost.com/2025/12/11/us-news/olivia-n...
58,52,2025-12-11-17-00-00 +0000,wsj,Two progressives competing in the Democratic p...,https://www.wsj.com/politics/elections/a-bible...
46,51,2025-12-11-17-27-39 +0000,nypost,Disney to let users make AI videos featuring b...,https://nypost.com/2025/12/11/business/disney-...
128,43,2025-12-11-13-29-48 +0000,nypost,Erika Kirk claps back at insane jewelry conspi...,https://nypost.com/2025/12/11/us-news/charlie-...
218,43,2025-12-11-05-52-19 +0000,bbc,BBC speaks to Nobel Peace Prize winner María C...,https://www.bbc.com/news/videos/cpwkvpr7n55o?a...
88,43,2025-12-11-15-40-06 +0000,nypost,CNN denies banning Stephen Miller after top WH...,https://nypost.com/2025/12/11/business/cnn-den...
260,40,2025-12-11-00-32-00 +0000,wsj,Fed Chair Jerome Powell said that Fed staffers...,https://www.wsj.com/economy/jobs/fed-chair-jer...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
